## Reading the pdf file

In [1]:
import pypdf

In [2]:
# read your file
file_path = './pdfs/testPart7.pdf'
# file_path = './pdfs/test.pdf'
with open(file_path, 'rb') as f:
    pdf = pypdf.PdfReader(f)

    # get the first page
    page = pdf.pages[0]

    # extract text from page
    text = page.extract_text()

    # do whatever you want with extracted text
    print(text)
text[772:786]

DETAIL DES PRESTATIONS (les minutes sont exprimées en centièmes d'heures) Quantité Taux Montant
HEURES TOTAL H.T TAUX T.V.A T.V.A TOTAL T.T.CTVA Intracommunautaire : FR18894239607ONE WORK                                                                    
211B AVENUE CHARLES DE                
92200      NEUILLY SUR SEINE               
SAS au capital de 100 000 euros                             
Garant Fin : GROUPAMA, 132 rue des 3 fontanot, Nanterre     Siret : 89423960700043 APE : 7820ZTel : 01 86 90 93 50      
Adresse de règlement :
 
 ORLY DISTRIBUTION                                                           
8 PLACE GASTON VIENS                  
94310      ORLY                            
AVOIR  Double Client : 010 110021 / CLI M S CTR -A       
AVOIR  010/1001076sur FACTURE N° 1000980 Date  : 31/12/2022 Page 2


'010/1001076sur'

In [3]:
# nlp = spacy.blank("en") 
# doc = nlp.make_doc(text)

# doc.char_span(811, 829, label='test', alignment_mode="contract")

In [4]:
start = [1, 2, 3, 811, 772, 772, 772, 772, 813, 813]
end = [1, 2, 3, 825, 786, 786, 786, 786, 824, 824]

## Creating dataset for the pdf file

In [5]:
import json

In [6]:
training_data = []
for i in range(3, 10):

    filePath = f'./pdfs/testPart{i}.pdf'
    # print(filePath)
    
    with open(filePath, 'rb') as f:
        pdf = pypdf.PdfReader(f)

        # get the first page
        page = pdf.pages[0]

        # extract text from page
        text = page.extract_text()

        # print(text)

        temp = {'text':text, 'entities': [(start[i], end[i], 'facture')]}

        training_data.append(temp)

# write to json file
# with open('training_data.json', 'w') as f:
#     json.dump(training_data, f)


In [7]:
# print(training_data)
training_data


[{'text': "DETAIL DES PRESTATIONS (les minutes sont exprimées en centièmes d'heures) Quantité Taux Montant\nHEURES TOTAL H.T TAUX T.V.A T.V.A TOTAL T.T.CTVA Intracommunautaire : FR18894239607ONE WORK                                                                    \n211B AVENUE CHARLES DE                \n92200      NEUILLY SUR SEINE               \nSAS au capital de 100 000 euros                             \nGarant Fin : GROUPAMA, 132 rue des 3 fontanot, Nanterre     Siret : 89423960700043 APE : 7820ZTel : 01 86 90 93 50      \nAdresse de règlement :\n \n BG APPRO                                                                    \n6 RUE DU CANAL                        \nZAC DE LA MARINIERE                   \n91070      BONDOUFLE                       \nAVOIR  Double Client : 010 110007 / CLI M S CTR -A       \nAVOIR  010/1000988sur FACTURE N° 1000770 Date  : 31/12/2022 Page 1\n                             \n              \n                        \n                               

## Changing training data to .spacy format

In [8]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [9]:

for training_example  in tqdm(training_data): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        # print(span, text[start:end])
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|██████████| 7/7 [00:00<00:00, 435.11it/s]


In [10]:
import spacy
from spacy import displacy

In [11]:
nlp = spacy.load("model-best")

In [12]:
file_path = './pdfs/testPart10.pdf'
# file_path = './pdfs/test.pdf'
with open(file_path, 'rb') as f:
    pdf = pypdf.PdfReader(f)

    # get the first page
    page = pdf.pages[0]

    # extract text from page
    text = page.extract_text()

doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

## Using Spacy

In [ ]:
import spacy
from spacy import displacy
from spacy import matcher

In [ ]:
# downloding spacy small english model
# !python -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

In [ ]:
# dispacy entity visualizer
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
# method extracts bill number using spacy matcher
def extract_bill_number(doc):
    # define the pattern
    pattern = [{'POS': 'PROPN'}, {'POS': 'NUM'}]

    # initialize matcher with a vocab
    Matcher = matcher(nlp.vocab)
    Matcher.add('BILL_NUMBER', None, pattern)

    # find matches in the doc
    matches = Matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text

In [ ]:
extract_bill_number(text)

In [ ]:
# method extracts client name from text
def extract_client_name(text):
    doc = nlp(text)
    res = []
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            res.append(ent.text)
    return res